# 10. 단어 Level로 번역기 업그레이드하기 

In [1]:
import os
import pandas as pd
import numpy as np
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

## 데이터 프레임

In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 197463


,eng,fra,cc
35845,I failed on purpose.,J'ai fait exprès d'échouer.,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
76694,You look beautiful to me.,Tu es belle à mes yeux.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
164014,Tom is very difficult to get along with.,Il est très difficile de s'entendre avec Tom.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
71588,I have certain standards.,J'ai certains principes.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
169686,There's a bus stop in front of our school.,Il y a un arrêt de bus devant notre école.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


## 데이터 전처리

In [3]:
lines = lines[['eng', 'fra']][:33000]
lines.sample(5)

,eng,fra
27776,You can't beat me.,Tu ne peux pas me battre.
159,Be calm.,Soyez calmes !
3525,Ignore them.,Ignorez-les.
8291,My hair's wet.,J'ai les cheveux mouillés.
16392,Tom seemed busy.,Tom avait l'air occupé.


## 정제화,정규화, 전처리

- 구두점 분리
- 소문자 변환
- 띄어쓰기 단위로 토큰화

In [4]:
def new_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    sentence = sentence.strip()
    sentence = sentence.split(" ")
    
    return sentence

In [5]:
def new_sentence_decoder(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    sentence = sentence.strip()
    sentence = ' ' + sentence + ' '
    sentence = sentence.split(" ")
    
    return sentence

In [6]:
lines.eng = lines.eng.apply(lambda x : new_sentence(x))
lines.fra = lines.fra.apply(lambda x : new_sentence_decoder(x))

In [7]:
lines.eng.sample(5)

15492     [people, like, tom, .]
4363          [you, re, rich, .]
21996    [who, ordered, that, ?]
14563     [i, was, convicted, .]
10999    [i, m, in, the, car, .]
Name: eng, dtype: object

In [8]:
lines.fra.sample(5)

27258    [, nous, nous, dirigeons, vers, le, nord, ., ]
32412               [, celui, ci, est, plus, gros, ., ]
25294           [, ne, laissez, personne, chapper, !, ]
19560                 [, je, n, ai, jamais, essay, ., ]
31819                       [, c, est, arriv, ici, ., ]
Name: fra, dtype: object

## 토크나이저로 텍스트 ㅡ> 숫자 변환

In [9]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(lines.eng)

In [10]:
input_text = eng_tokenizer.texts_to_sequences(lines.eng)
input_text[:5]

[[28, 1], [28, 1], [28, 1], [28, 1], [761, 1]]

In [12]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)
target_text[:5]

[[1, 74, 7, 1],
 [1, 364, 2, 1],
 [1, 27, 511, 7, 1],
 [1, 715, 7, 1],
 [1, 749, 7, 1]]

In [13]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4671
프랑스어 단어장의 크기 : 7453


In [14]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [15]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4671
프랑스어 단어장의 크기 : 7453
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


## Teacher forcing

In [17]:
sos_token = ''
eos_token = ''

encoder_input = input_text

decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 

decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]


In [18]:
print(decoder_input[:3])
print(decoder_target[:3])

[[74, 7], [364, 2], [27, 511, 7]]
[[74, 7], [364, 2], [27, 511, 7]]


## Padding

In [19]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))


영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


## Validation

In [20]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 17)
(30000, 17)
(3000, 8)
(3000, 17)
(3000, 17)


## Encoder

In [21]:
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, 256, input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(units = 256, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

## Decoder


In [22]:
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(fra_vocab_size, 256)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
decoder_outputs, _, _= decoder_lstm(dec_masking, initial_state = encoder_states)
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

## Model

In [23]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    1195776     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    1907968     input_2[0][0]                    
______________________________________________________________________________________________

In [24]:
model.compile(optimizer="rmsprop", loss='sparse_categorical_crossentropy', metrics=['acc'])


In [26]:
model.fit([encoder_input_train, decoder_input_train],decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 32, epochs = 300 )

Epoch 1/300
938/938 [==============================] - 15s 16ms/step - loss: 0.4240 - acc: 0.9468 - val_loss: 0.3652 - val_acc: 0.9560
Epoch 2/300
938/938 [==============================] - 15s 16ms/step - loss: 0.3280 - acc: 0.9606 - val_loss: 0.3123 - val_acc: 0.9640
Epoch 3/300
938/938 [==============================] - 15s 16ms/step - loss: 0.2848 - acc: 0.9670 - val_loss: 0.2785 - val_acc: 0.9688
Epoch 4/300
938/938 [==============================] - 15s 16ms/step - loss: 0.2554 - acc: 0.9713 - val_loss: 0.2609 - val_acc: 0.9724
Epoch 5/300
938/938 [==============================] - 15s 16ms/step - loss: 0.2370 - acc: 0.9742 - val_loss: 0.2448 - val_acc: 0.9747
Epoch 6/300
938/938 [==============================] - 15s 16ms/step - loss: 0.2189 - acc: 0.9767 - val_loss: 0.2302 - val_acc: 0.9766
Epoch 7/300
938/938 [==============================] - 15s 16ms/step - loss: 0.2031 - acc: 0.9786 - val_loss: 0.2187 - val_acc: 0.9783
Epoch 8/300
938/938 [==============================] - 

## Encoder

In [27]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         1195776   
_________________________________________________________________
masking (Masking)            (None, None, 256)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 525312    
Total params: 1,721,088
Trainable params: 1,721,088
Non-trainable params: 0
_________________________________________________________________


## Decoder

In [28]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_emb2 = Embedding(fra_vocab_size, 256)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [29]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    1907968     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
____________________________________________________________________________________________

In [30]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [31]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # 에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # 에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [32]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [33]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['']) and i!=fra2idx['']):
            temp = temp + idx2fra[i] + ' '
    return temp

In [38]:
for seq_index in [1,100,301,777,2222]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])


-----------------------------------
입력 문장: stop squabbling . 
정답 문장: arr tez de vous chamailler . 
번역기가 번역한 문장:  uvre t vot est es
-----------------------------------
입력 문장: it s your bedtime . 
정답 문장: c est l heure d aller te coucher . 
번역기가 번역한 문장:  vous est positio
-----------------------------------
입력 문장: he writes arabic . 
정답 문장: il crit arabe . 
번역기가 번역한 문장:  d hurl croire croir
-----------------------------------
입력 문장: they re fearless . 
정답 문장: elles sont intr pides . 
번역기가 번역한 문장:  vous der entr ent
-----------------------------------
입력 문장: you crack me up . 
정답 문장: tu me fais mourir de rire . 
번역기가 번역한 문장:  entr fortement enduran


- epoch를 300을 주고 돌려보았지만, 50번대보다 정확성이 낮아졌다. 욕심만 과했던것 같다. 그래서 결과도 좋지 않은 듯 하다.